In [1]:
# google drive mount

from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
!pip install streamlit -q
!pip install pyngrok

     |████████████████████████████████| 9.2 MB 4.6 MB/s 
     |████████████████████████████████| 164 kB 59.8 MB/s 
     |████████████████████████████████| 4.7 MB 59.2 MB/s 
     |████████████████████████████████| 78 kB 4.0 MB/s 
     |████████████████████████████████| 182 kB 19.6 MB/s 
     |████████████████████████████████| 237 kB 54.0 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 51 kB 6.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 7.9 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=32192782063f3f17f476aab8ad21dd75c5ab1528b0d958869c735e8d7b47e1f6
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [3]:
from pyngrok import ngrok

ngrok.set_auth_token('2BVeByVJG3w1QaQFmdREM9308C0_4V7edUh4hAybvpQ5dqRCn')

In [4]:
!pip install gluonnlp==0.10.0 pandas tqdm   
!pip install mxnet
!pip install sentencepiece==0.1.91
# !pip install transformers==4.8.2
!pip install torch==1.8.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 8.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595741 sha256=72adb8bbb39562d173d6f6d47fec46ce1eb3f0186816a9e394abb16cc3e75162
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 1.5 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 7.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg

In [5]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-572rd8kf
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-572rd8kf
     |████████████████████████████████| 129 kB 7.7 MB/s 
     |████████████████████████████████| 54.7 MB 17 kB/s 
     |████████████████████████████████| 4.5 MB 36.6 MB/s 
     |████████████████████████████████| 2.5 MB 38.7 MB/s 
     |████████████████████████████████| 73 kB 1.7 MB/s 
     |████████████████████████████████| 6.7 MB 31.6 MB/s 
     |████████████████████████████████| 3.3 MB 46.4 MB/s 
     |████████████████████████████████| 880 kB 48.0 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15708 sha256=e6cffbb46a931fe10a684d65ea378f63ad64f618bb525a760de2d80186bdd6b0
  Stored in directory: /tmp/pip-ephem-wheel-cache-xavb0427/wheels/d3/68/ca/33474

In [6]:
!pip install git+https://github.com/ssut/py-hanspell.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-1lpiw94d
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-1lpiw94d
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4869 sha256=b803ce4f222344f2e8cd027eb20a3ad5cc7e69e8c50c5f716e5746a6b8799dba
  Stored in directory: /tmp/pip-ephem-wheel-cache-_7z62p6c/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


In [7]:
%%writefile app.py

import streamlit as st
from hanspell import spell_checker
import re
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


# tokenizing
class EmotionDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
    transform = nlp.data.BERTSentenceTransform(bert_tokenizer, 
                                               max_seq_length=max_len, pad=pad, pair=pair) 

    self.sentences = [transform([i[sent_idx]]) for i in dataset]
    self.labels = [np.int32(i[label_idx]) for i in dataset]

  def __getitem__(self, i):
    return (self.sentences[i] + (self.labels[i], ))

  def __len__(self):
    return (len(self.labels))


# classifier
class EmotionClassifier(nn.Module):
  def __init__(self,
               bert,
               hidden_size = 768,
               num_classes = 9,
               dr_rate = None,
               params = None):
    super(EmotionClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    self.classifier = nn.Linear(hidden_size, num_classes)
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)

  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    return attention_mask.float()

  def forward(self, token_ids, valid_length, segment_ids):
    attention_mask = self.gen_attention_mask(token_ids, valid_length)
    
    _, pooler = self.bert(input_ids=token_ids, token_type_ids=segment_ids.long(), 
                          attention_mask=attention_mask.float().to(token_ids.device), return_dict=False)
    
    if self.dr_rate:
      out = self.dropout(pooler)
    
    return self.classifier(out)


@st.cache(allow_output_mutation=True)
def loading_model():
    bertmodel, vocab = get_pytorch_kobert_model()

    tokenizer = get_tokenizer()
    tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

    return tok


def cleaning(predict_sentence):

    # 숫자, 문자 아닌 글자 제거
    removed_sent = re.sub('[^\w\s]', '', predict_sentence).strip()

    # 맞춤법 교정
    spelled_sent = spell_checker.check(removed_sent)
    checked_sent = spelled_sent.checked

    return checked_sent


def predict(predict_sentence, model):

    max_len = 64
    batch_size = 32

    # flipped emotion dict
    emotion_labels = {'중립': 0,
                      '기쁨': 1,
                      '놀람': 2,
                      '긴장됨': 3,
                      '괴로움': 4,
                      '화남': 5,
                      '비참함': 6,
                      '우울함': 7,
                      '피로함': 8}
    flip_emotion_labels = {v: k for k, v in emotion_labels.items()}

    data =[predict_sentence, '0']
    dataset_new = [data]

    tok = loading_model()

    new_test = EmotionDataset(dataset_new, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(new_test, batch_size=batch_size, num_workers=5)

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out1 = model(token_ids, valid_length, segment_ids)
        for i in out1:
            logits = i
            logits = logits.detach().cpu().numpy()

            logits_zero = [np.round(x, 3) if x > 0 else 0 for x in logits]
            # print(logits_zero, flip_emotion_labels[np.argmax(logits)])

            return flip_emotion_labels[np.argmax(logits)]


@st.cache(allow_output_mutation=True)
def load_kobert_model():
    # load model
    model = torch.load('/gdrive/My Drive/KoBERT/checkpoint/kobert_v2.pt', map_location=device)
    model.load_state_dict(torch.load('/gdrive/My Drive/KoBERT/checkpoint/kobert_v2_state_dict.pt', map_location=device))
    model.eval()

    return model


if __name__ == '__main__':

  kobert_model = load_kobert_model()

  st.markdown("<h1 style='text-align: center;'>BERT 기반 감정 분석 모델</h1>", unsafe_allow_html=True)
  st.write('\n\n\n')
  input_sent = st.text_input('분석하고 싶은 문장을 입력해주세요!', '')

  if input_sent is None:
      st.text('문장을 입력해주세요')
  else:
      st.write('\n\n\n')
      st.subheader('감정 분석 결과')
      emotion = st.text('...')
      pred = predict(cleaning(input_sent), kobert_model)
      emotion.text(pred)


Writing app.py


In [15]:
!ls

app.py	nohup.out  sample_data


In [16]:
!cat app.py


import streamlit as st
from hanspell import spell_checker
import re
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


# tokenizing
class EmotionDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
    transform = nlp.data.BERTSentenceTransform(bert_tokenizer, 
                                               max_seq_length=max_len, pad=pad, pair=pair) 

    self.sentences = [transform([i[sent_idx]]) for i in dataset]
    self.labels = [np.int32(i[label_idx]) for i in dataset]

  def __getitem__(self, i):
    return (self.sentences[i] + (self.labels[i], ))

  def __len__(self):
    return (len(self.labels))


# classifier
class EmotionClassifier(nn.Module):
  def __init__

In [17]:
!nohup streamlit run app.py --server.port 80 &

nohup: appending output to 'nohup.out'


In [18]:
url = ngrok.connect(port='80')
url

<NgrokTunnel: "http://024a-34-125-236-105.ngrok.io" -> "http://localhost:80">

In [14]:
ngrok.kill()